문제 : https://dacon.io/competitions/official/235747/overview/description

idea

1) KoBERT 를 통한 분류

2) keyword 추출 후 clustering을 통한 분류

2 번을 사용하면 입력 토큰 수를 크게 주일 수 있을 듯 함.

# import

In [ ]:
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 30.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=619630 sha256=3bc0b720ad343217daa1ae52f2fe9a40051e77b531820a9e8ee40703224659d0
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook


# EDA

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/personal/news_topic_classification/data/train_data.csv")
test = pd.read_csv("/content/drive/MyDrive/personal/news_topic_classification/data/test_data.csv")

In [ ]:
len(train)

45654

한자가 있음. KoBERT 사용을 위해 한자를 한국어로 변환.

In [ ]:
ch = re.compile('[一-龥]')

In [ ]:
ch_dic = {}
for i in range(len(train)):
  temp = ch.findall(train.loc[i,'title'])
  if len(temp) != 0:
    for item in temp:
      if item in ch_dic:
        ch_dic[item] += 1
      else:
        ch_dic[item] = 1


In [ ]:
ch_dic = sorted(ch_dic.items(), key=lambda x: x[1], reverse=True)

In [ ]:
ch_dic

[('美', 1498),
 ('北', 1329),
 ('中', 795),
 ('朴', 661),
 ('日', 467),
 ('靑', 381),
 ('與', 291),
 ('英', 285),
 ('文', 184),
 ('野', 181),
 ('獨', 138),
 ('伊', 127),
 ('韓', 109),
 ('前', 95),
 ('佛', 92),
 ('檢', 73),
 ('軍', 69),
 ('安', 63),
 ('反', 54),
 ('行', 44),
 ('南', 37),
 ('硏', 27),
 ('故', 25),
 ('外', 24),
 ('亞', 23),
 ('對', 21),
 ('銀', 19),
 ('展', 19),
 ('重', 18),
 ('株', 18),
 ('新', 17),
 ('黃', 16),
 ('企', 14),
 ('詩', 14),
 ('車', 14),
 ('孫', 14),
 ('親', 13),
 ('黨', 13),
 ('人', 13),
 ('比', 12),
 ('金', 12),
 ('號', 12),
 ('曺', 11),
 ('禹', 11),
 ('男', 11),
 ('無', 9),
 ('秋', 9),
 ('潘', 8),
 ('印', 8),
 ('駐', 8),
 ('弗', 8),
 ('發', 7),
 ('先', 7),
 ('法', 7),
 ('證', 7),
 ('千', 7),
 ('委', 7),
 ('大', 7),
 ('家', 6),
 ('百', 6),
 ('風', 6),
 ('社', 5),
 ('月', 5),
 ('崔', 4),
 ('心', 4),
 ('後', 4),
 ('色', 4),
 ('州', 4),
 ('分', 3),
 ('高', 3),
 ('核', 3),
 ('非', 3),
 ('全', 3),
 ('脫', 3),
 ('丁', 3),
 ('代', 3),
 ('父', 3),
 ('史', 3),
 ('惡', 2),
 ('告', 2),
 ('富', 2),
 ('案', 2),
 ('院', 2),
 ('和', 2),
 ('尼', 2),
 ('燈'

미리 한자어 뜻을 찾아 둔 분이 있었음.
https://dacon.io/competitions/official/235747/codeshare/3054?page=1&dtype=recent

In [ ]:
def ch_to_kor(dataset):
  '''
  dataset을 입력하면 해당 'title'에 있는 한자를 한국어로 바꿔서 반환
  '''
  ch_map = { "中": "중국", "美": "미국","北":"북한",'日':"일본",'英':'영국','行':'행','靑':'청와대','朴':'박','銀':'은행','與':'여당',
     '文':'문','野':'야당','獨':'독일','伊':'이탈리아','韓':'한국','佛':'프랑스','前':'전','檢':'검찰','軍':'군','安':'안철수','南':'남한',
     '亞':'아시아','展':'전시회','重':'차','株':'주식','詩':'시'}
  
  for i in range(len(dataset)):
    for ch, kor in ch_map.items():
      if ch in dataset.loc[i,'title']:
        dataset.loc[i,'title'] = dataset.loc[i,'title'].replace(ch, kor)

  return dataset

In [ ]:
train = ch_to_kor(train)
test = ch_to_kor(test)

In [ ]:
train

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 미국전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근한국기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
...,...,...,...
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1
45652,45652,답변하는 배기동 국립중앙박물관장,2


In [ ]:
train_dataset = train[:3*(len(train)//4)]
valid_dataset = train[3*(len(train)//4):]

In [ ]:
train_dataset

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 미국전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근한국기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
...,...,...,...
34234,34234,언론자유의 상징 굽히지 않는 펜,2
34235,34235,중국 미국 에너지 수입 늘리지도 줄이지도 못할 처지,4
34236,34236,동양피엔에프 60억원 규모 공급 계약,1
34237,34237,LG V30 10월 5일 미국 출시…픽셀2와 쌍끌이 흥행 노린다,0


In [ ]:
test

,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다
3,45657,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4,45658,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간
...,...,...
9126,54780,인천 오후 3시35분 대설주의보…눈 3.1cm 쌓여
9127,54781,노래방에서 지인 성추행 외교부 사무관 불구속 입건종합
9128,54782,40년 전 부마항쟁 부산 시위 사진 2점 최초 공개
9129,54783,게시판 아리랑TV 아프리카개발은행 총회 개회식 생중계


# Model load


참고 : https://velog.io/@jiyoung/Text-ClassificationKoBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0-%EC%BD%94%EB%93%9C

## KoBERT load

In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git

Cloning into 'KoBERT'...
remote: Enumerating objects: 428, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 428 (delta 125), reused 104 (delta 102), pack-reused 280
Receiving objects: 100% (428/428), 218.85 KiB | 12.16 MiB/s, done.
Resolving deltas: 100% (221/221), done.


In [ ]:
cd KoBERT

/content/KoBERT


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 8.5 MB/s 
     |████████████████████████████████| 54.7 MB 40 kB/s 
     |████████████████████████████████| 4.5 MB 63.2 MB/s 
     |████████████████████████████████| 1.2 MB 61.1 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x3952c000 @  0x7fc6ac207615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |████████████████████████████████| 881.9 MB 8.5 kB/s 
     |████████████████████████████████| 2.5 MB 63.4 MB/s 
     |████████████████████████████████| 73 kB 2.4 MB/s 
     |████████████████████████████████| 6.7 MB 65.1 MB/s 
     |████████████████████████

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

## model 

gluonnlp 공식 문석 참고 : https://nlp.gluon.ai/api/data.html

In [ ]:
class BERTDataset(Dataset):
  # dataframe 형태로 넣어주기 위해 sent_idx 를 sent_col로 바꿈.
    def __init__(self, dataset, sent_col, label_col, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_col]]) for idx, i in dataset.iterrows()]
        self.labels = [np.int32(i[label_col]) for idx, i in dataset.iterrows()]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/KoBERT/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#tokenize
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(train_dataset, 'title', 'topic_idx', tok, max_len, True, False)
data_valid = BERTDataset(valid_dataset, 'title', 'topic_idx', tok, max_len, True, False)


using cached model. /content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# data loader
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=2)


In [ ]:
'''
# data loader 확인
for token_ids, valid_length, segment_ids, label in train_dataloader:
  print(label)
  break
'''

'\n# data loader 확인\nfor token_ids, valid_length, segment_ids, label in train_dataloader:\n  print(label)\n  break\n'

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7, 
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) #gpu

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    valid_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(valid_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        valid_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, valid_acc / (batch_id+1)))

<ipython-input-48-9af2ab371229>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1070 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9693044424057007 train acc 0.15625
epoch 1 batch id 201 loss 1.0879621505737305 train acc 0.435634328358209
epoch 1 batch id 401 loss 0.9676445722579956 train acc 0.6249220698254364
epoch 1 batch id 601 loss 0.6996781826019287 train acc 0.6902038269550749
epoch 1 batch id 801 loss 0.38375145196914673 train acc 0.7259285268414482
epoch 1 batch id 1001 loss 0.1350795328617096 train acc 0.7651723276723277
epoch 1 train acc 0.7775964727163099


<ipython-input-48-9af2ab371229>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(valid_dataloader)):


  0%|          | 0/357 [00:00<?, ?it/s]

...

epoch 4 validation acc 0.8717612044817927


  0%|          | 0/1070 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2909173369407654 train acc 0.90625
epoch 5 batch id 201 loss 0.10968850553035736 train acc 0.9544465174129353
epoch 5 batch id 401 loss 0.199753075838089 train acc 0.9555018703241895
epoch 5 batch id 601 loss 0.024292632937431335 train acc 0.9601185524126455
epoch 5 batch id 801 loss 0.036983393132686615 train acc 0.9627418851435705
epoch 5 batch id 1001 loss 0.020300017669796944 train acc 0.9679695304695305
epoch 5 train acc 0.969713785046729


  0%|          | 0/357 [00:00<?, ?it/s]

epoch 5 validation acc 0.8799894957983193


In [ ]:
class BERTDataset_test(Dataset):
  # dataframe 형태로 넣어주기 위해 sent_idx 를 sent_col로 바꿈.
    def __init__(self, dataset, sent_col,  bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.len = len(dataset)
        self.sentences = [transform([i[sent_col]]) for idx, i in dataset.iterrows()]

    def __getitem__(self, i):
        return self.sentences[i]

    def __len__(self):
        return self.len

In [ ]:
data_test = BERTDataset_test(test, 'title', tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=1, num_workers=2)


In [ ]:
output = []
for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  output.append(prediction)




<ipython-input-60-5573a5b0d0ae>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/9131 [00:00<?, ?it/s]

In [ ]:
submit = pd.read_csv("/content/drive/MyDrive/personal/news_topic_classification/data/sample_submission.csv", index_col=False)


In [ ]:
submit['topic_idx'] = output

In [ ]:
submit.to_csv("/content/drive/MyDrive/personal/news_topic_classification/data/submission.csv", index=False)

In [ ]:
test

,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다
3,45657,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4,45658,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간
...,...,...
9126,54780,인천 오후 3시35분 대설주의보…눈 3.1cm 쌓여
9127,54781,노래방에서 지인 성추행 외교부 사무관 불구속 입건종합
9128,54782,40년 전 부마항쟁 부산 시위 사진 2점 최초 공개
9129,54783,게시판 아리랑TV 아프리카개발은행 총회 개회식 생중계


In [ ]:
submit

,index,topic_idx
0,45654,0
1,45655,3
2,45656,2
3,45657,2
4,45658,3
...,...,...
9126,54780,3
9127,54781,2
9128,54782,2
9129,54783,2


# validation set을 없애고 training

In [ ]:
train_dataset2 = train
data_train2 = BERTDataset(train_dataset2, 'title', 'topic_idx', tok, max_len, True, False)
train_dataloader2 = torch.utils.data.DataLoader(data_train2, batch_size=batch_size, num_workers=2)


In [ ]:
model2 = BERTClassifier(bertmodel, dr_rate=0.5).to(device) #gpu

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters2 = [
    {'params': [p for n, p in model2.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model2.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer2 = AdamW(optimizer_grouped_parameters2, lr=learning_rate)
loss_fn2 = nn.CrossEntropyLoss()

t_total2 = len(train_dataloader2) * num_epochs
warmup_step2 = int(t_total2 * warmup_ratio)

scheduler2 = get_cosine_schedule_with_warmup(optimizer2, num_warmup_steps=warmup_step2, num_training_steps=t_total2)

In [ ]:
num_epochs2 = 5

In [ ]:
for e in range(num_epochs2):
    train_acc2 = 0.0
    model2.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader2)):
        optimizer2.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model2(token_ids, valid_length, segment_ids)
        loss2 = loss_fn2(out, label)
        loss2.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), max_grad_norm)
        optimizer2.step()
        scheduler2.step()  # Update learning rate schedule
        train_acc2 += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss2 {} train acc {}".format(e+1, batch_id+1, loss2.data.cpu().numpy(), train_acc2 / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc2 / (batch_id+1)))
   

<ipython-input-44-f36d99cf5180>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader2)):


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 1 batch id 1 loss2 1.8099868297576904 train acc 0.1875
epoch 1 batch id 201 loss2 0.537433922290802 train acc 0.6422574626865671
epoch 1 batch id 401 loss2 0.4990735352039337 train acc 0.800498753117207
epoch 1 batch id 601 loss2 0.052143536508083344 train acc 0.8570611480865225
epoch 1 batch id 801 loss2 0.06042325496673584 train acc 0.8831148564294632
epoch 1 batch id 1001 loss2 0.005022981204092503 train acc 0.9030344655344655
epoch 1 batch id 1201 loss2 0.17851857841014862 train acc 0.9167360532889259
epoch 1 batch id 1401 loss2 0.02471558377146721 train acc 0.9236482869379015
epoch 1 train acc 0.9248204274702172


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 2 batch id 1 loss2 0.0040147388353943825 train acc 1.0
epoch 2 batch id 201 loss2 0.027097051963210106 train acc 0.959110696517413
epoch 2 batch id 401 loss2 0.34849467873573303 train acc 0.9609569825436409
epoch 2 batch id 601 loss2 0.2843030095100403 train acc 0.9621464226289518
epoch 2 batch id 801 loss2 0.2552112936973572 train acc 0.9624687890137328
epoch 2 batch id 1001 loss2 0.001731764990836382 train acc 0.96747002997003
epoch 2 batch id 1201 loss2 0.00797424465417862 train acc 0.9705713988343048
epoch 2 batch id 1401 loss2 0.003093262668699026 train acc 0.9710251605995718
epoch 2 train acc 0.9714873861247372


...

epoch 3 batch id 1 loss2 0.007958592846989632 train acc 1.0
epoch 3 batch id 201 loss2 0.04933390021324158 train acc 0.9704601990049752
epoch 3 batch id 401 loss2 0.22659610211849213 train acc 0.9695293017456359
epoch 3 batch id 601 loss2 0.19240280985832214 train acc 0.9727537437603994
epoch 3 batch id 801 loss2 0.10655336081981659 train acc 0.9738998127340824
epoch 3 batch id 1001 loss2 0.0012411904754117131 train acc 0.9771166333666333
epoch 3 batch id 1201 loss2 0.0012023249873891473 train acc 0.979860532889259
epoch 3 batch id 1401 loss2 0.0011134520173072815 train acc 0.981307994289793
epoch 3 train acc 0.981582866152768


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 4 batch id 1 loss2 0.002248810138553381 train acc 1.0
epoch 4 batch id 201 loss2 0.0063438634388148785 train acc 0.9818097014925373
epoch 4 batch id 401 loss2 0.11149486154317856 train acc 0.98285536159601
epoch 4 batch id 601 loss2 0.00365948467515409 train acc 0.9849209650582362
epoch 4 batch id 801 loss2 0.000972650246694684 train acc 0.9856429463171036
epoch 4 batch id 1001 loss2 0.0005999119020998478 train acc 0.9875749250749251
epoch 4 batch id 1201 loss2 0.0008571000653319061 train acc 0.9889415070774354
epoch 4 batch id 1401 loss2 0.0005972072831355035 train acc 0.9897617773019272
epoch 4 train acc 0.9898826208829713


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 5 batch id 1 loss2 0.18841715157032013 train acc 0.96875
epoch 5 batch id 201 loss2 0.0009552127448841929 train acc 0.9869402985074627
epoch 5 batch id 401 loss2 0.001397064421325922 train acc 0.9884663341645885
epoch 5 batch id 601 loss2 0.004195230547338724 train acc 0.990484608985025
epoch 5 batch id 801 loss2 0.0005850218585692346 train acc 0.9915340199750312
epoch 5 batch id 1001 loss2 0.0005884908605366945 train acc 0.9926948051948052
epoch 5 batch id 1201 loss2 0.02176087722182274 train acc 0.9932868442964197
epoch 5 batch id 1401 loss2 0.0007140305242501199 train acc 0.9937990720913633
epoch 5 train acc 0.9939120532585844


In [ ]:
output2 = []
for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model2(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  output2.append(prediction)



<ipython-input-45-ab172225c8be>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/9131 [00:00<?, ?it/s]

In [ ]:
submit2 = pd.read_csv("/content/drive/MyDrive/personal/news_topic_classification/data/sample_submission.csv", index_col=False)
submit2['topic_idx'] = output2
submit2.to_csv("/content/drive/MyDrive/personal/news_topic_classification/data/submission_5epochs.csv", index=False)

# keyword 추출

a . 영어 번역 후 keybert 사용

b. keybert 에 kobert_tokenizer를 잘 넣으면 ?


이후 클러스터링


In [ ]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.0 MB/s 
     |████████████████████████████████| 237 kB 48.1 MB/s 
     |████████████████████████████████| 51 kB 1.7 MB/s 
     |████████████████████████████████| 5.8 MB 46.8 MB/s 
     |████████████████████████████████| 1.3 MB 20.0 MB/s 
     |████████████████████████████████| 182 kB 66.8 MB/s 
     |████████████████████████████████| 7.6 MB 50.6 MB/s 
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23800 sha256=db5b43cfdf8f3d6f1aa5780ef1984a3b0ccef634c653c6deab3606deba8db304
  Stored in directory: /root/.cache/pip/wheels/6c/bc/8b/a51bee77aec33895e6c8c236144b4cc10875659c4d2c80f070
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=32563ea2402b426adb21da93b883335f54d4a7139587f6de2ef24d1d39a8b9d0
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88

In [ ]:
from keybert import KeyBERT

In [ ]:
doc = "인천→핀란드 항공기 결항…휴가철 여행객 분통	"
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
doc = "인천 핀란드 항공기 결항…휴가철 여행객 분통"
keywords = kw_model.extract_keywords(doc)

In [ ]:
keywords

[('여행객', 0.5427),
 ('결항', 0.4649),
 ('항공기', 0.4384),
 ('인천', 0.4107),
 ('휴가철', 0.3684)]

In [ ]:
te= []
te.extend(["tset", "dsf", "cds"])

In [ ]:
te

['tset', 'dsf', 'cds']

In [ ]:
train

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 미국전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근한국기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
...,...,...,...
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1
45652,45652,답변하는 배기동 국립중앙박물관장,2


In [ ]:
train

,index,title,topic_idx,key_words
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 미국전역 거점화,4,
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,
3,3,NYT 클린턴 측근한국기업 특수관계 조명…공과 사 맞물려종합,4,
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,
...,...,...,...,...
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1,
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2,
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1,
45652,45652,답변하는 배기동 국립중앙박물관장,2,


In [ ]:
result = []
for idx, row in train.iterrows():
  temp_key = kw_model.extract_keywords(row['title'])
  try:
    result.append([temp_key[0][0], temp_key[1][0], temp_key[2][0]])
  except:
    try: 
      result.append([temp_key[0][0], temp_key[1][0]])
      print(result.append([temp_key[0][0], temp_key[1][0], temp_key[2][0]]))
    except:
      break
 


In [ ]:
temp